##### Copyright 2019 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Pix2Pix

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/generative/pix2pix"><img src="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/tutorials/generative/pix2pix.ipynb"> TensorFlow.orgで表示</a></td>
  <td>
<img src="https://www.tensorflow.org/images/colab_logo_32px.png"><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tutorials/generative/pix2pix.ipynb">Google Colab で実行</a>
</td>
  <td>
<img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"><a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/tutorials/generative/pix2pix.ipynb">GitHub でソースを表示</a>
</td>
  <td>
<img src="https://www.tensorflow.org/images/download_logo_32px.png"><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/generative/pix2pix.ipynb">ノートブックをダウンロード</a>
</td>
</table>

このノートブックでは、「[条件付き敵対的ネットワークを使用した瓦当から画像への変換](https://arxiv.org/abs/1611.07004)」で説明された条件付き GAN を使用して画像から画像への変換を実演します。このテクニックを使用すると、モノクロ写真のカラー画像化やGoogle マップから Google Earth への変換などを実行できるようになります。ここでは、建物のファサードを実際の建物に変換します。

例では、[プラハにあるチェコ工科大学](https://www.cvut.cz/)の[機械知覚センター](http://cmp.felk.cvut.cz/) が提供する [CMP Facade Database](http://cmp.felk.cvut.cz/~tylecr1/facade/) を使用します。この例を手短に紹介できるように、上述の[論文](https://arxiv.org/abs/1611.07004)の執筆者が作成した、このデータセットの事前処理済みの[コピー](https://people.eecs.berkeley.edu/~tinghuiz/projects/pix2pix/datasets/)を使用することにします。

各エポックには、単一 V100 GPU で約 15 秒がかかります。

以下は、モデルを 200 エポック、トレーニングした後に生成された出力です。

![sample output_1](https://www.tensorflow.org/images/gan/pix2pix_1.png) ![sample output_2](https://www.tensorflow.org/images/gan/pix2pix_2.png)

## TensorFlow とその他のライブラリをインポートする

In [ ]:
import tensorflow as tf

import os
import time

from matplotlib import pyplot as plt
from IPython import display

In [ ]:
!pip install -U tensorboard

## データセットを読み込む

このデータセットと、これに類似するデータセットは[こちら](https://people.eecs.berkeley.edu/~tinghuiz/projects/pix2pix/datasets)からダウントードできます。[論文](https://arxiv.org/abs/1611.07004)に書かれているように、子のトレーニングデータセットにランダムジッタリングとミラーリングを適用します。

- ランダムジッタリングでは、画像サイズは `286 x 286` に変換されてから、`256 x 256` にランダムにクロップされます。
- ランダムミラーリングでは、画像はランダムに水平に反転（左から右）されます。

In [ ]:
_URL = 'https://people.eecs.berkeley.edu/~tinghuiz/projects/pix2pix/datasets/facades.tar.gz'

path_to_zip = tf.keras.utils.get_file('facades.tar.gz',
                                      origin=_URL,
                                      extract=True)

PATH = os.path.join(os.path.dirname(path_to_zip), 'facades/')

In [ ]:
BUFFER_SIZE = 400
BATCH_SIZE = 1
IMG_WIDTH = 256
IMG_HEIGHT = 256

In [ ]:
def load(image_file):
  image = tf.io.read_file(image_file)
  image = tf.image.decode_jpeg(image)

  w = tf.shape(image)[1]

  w = w // 2
  real_image = image[:, :w, :]
  input_image = image[:, w:, :]

  input_image = tf.cast(input_image, tf.float32)
  real_image = tf.cast(real_image, tf.float32)

  return input_image, real_image

In [ ]:
inp, re = load(PATH+'train/100.jpg')
# casting to int for matplotlib to show the image
plt.figure()
plt.imshow(inp/255.0)
plt.figure()
plt.imshow(re/255.0)

In [ ]:
def resize(input_image, real_image, height, width):
  input_image = tf.image.resize(input_image, [height, width],
                                method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
  real_image = tf.image.resize(real_image, [height, width],
                               method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

  return input_image, real_image

In [ ]:
def random_crop(input_image, real_image):
  stacked_image = tf.stack([input_image, real_image], axis=0)
  cropped_image = tf.image.random_crop(
      stacked_image, size=[2, IMG_HEIGHT, IMG_WIDTH, 3])

  return cropped_image[0], cropped_image[1]

In [ ]:
# normalizing the images to [-1, 1]

def normalize(input_image, real_image):
  input_image = (input_image / 127.5) - 1
  real_image = (real_image / 127.5) - 1

  return input_image, real_image

In [ ]:
@tf.function()
def random_jitter(input_image, real_image):
  # resizing to 286 x 286 x 3
  input_image, real_image = resize(input_image, real_image, 286, 286)

  # randomly cropping to 256 x 256 x 3
  input_image, real_image = random_crop(input_image, real_image)

  if tf.random.uniform(()) &gt; 0.5:
    # random mirroring
    input_image = tf.image.flip_left_right(input_image)
    real_image = tf.image.flip_left_right(real_image)

  return input_image, real_image

以下の画像に見られるとおり、以下を行うために論文に説明されているようにランダムジッタリングを適用しています。

1. 画像サイズの高さと幅を大きくする
2. ターゲットサイズにランダムにクロップする
3. 画像を水平方向にランダムに反転させる

In [ ]:
plt.figure(figsize=(6, 6))
for i in range(4):
  rj_inp, rj_re = random_jitter(inp, re)
  plt.subplot(2, 2, i+1)
  plt.imshow(rj_inp/255.0)
  plt.axis('off')
plt.show()

In [ ]:
def load_image_train(image_file):
  input_image, real_image = load(image_file)
  input_image, real_image = random_jitter(input_image, real_image)
  input_image, real_image = normalize(input_image, real_image)

  return input_image, real_image

In [ ]:
def load_image_test(image_file):
  input_image, real_image = load(image_file)
  input_image, real_image = resize(input_image, real_image,
                                   IMG_HEIGHT, IMG_WIDTH)
  input_image, real_image = normalize(input_image, real_image)

  return input_image, real_image

## 入力パイプライン

In [ ]:
train_dataset = tf.data.Dataset.list_files(PATH+'train/*.jpg')
train_dataset = train_dataset.map(load_image_train,
                                  num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE)

In [ ]:
test_dataset = tf.data.Dataset.list_files(PATH+'test/*.jpg')
test_dataset = test_dataset.map(load_image_test)
test_dataset = test_dataset.batch(BATCH_SIZE)

## ジェネレータを構築する

- ジェネレータのアーキテクチャは、変更された U-Net です。
- エンコーダの各ブロックは (Conv -> Batchnorm -> Leaky ReLU) です。
- デコーダの各ブロックは (Transposed Conv -> Batchnorm -> Dropout(applied to the first 3 blocks) -> ReLU) です。
- エンコーダとデコーダ間にはスキップ接続があります（U-Net）。


In [ ]:
OUTPUT_CHANNELS = 3

In [ ]:
def downsample(filters, size, apply_batchnorm=True):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
      tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False))

  if apply_batchnorm:
    result.add(tf.keras.layers.BatchNormalization())

  result.add(tf.keras.layers.LeakyReLU())

  return result

In [ ]:
down_model = downsample(3, 4)
down_result = down_model(tf.expand_dims(inp, 0))
print (down_result.shape)

In [ ]:
def upsample(filters, size, apply_dropout=False):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
    tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                    padding='same',
                                    kernel_initializer=initializer,
                                    use_bias=False))

  result.add(tf.keras.layers.BatchNormalization())

  if apply_dropout:
      result.add(tf.keras.layers.Dropout(0.5))

  result.add(tf.keras.layers.ReLU())

  return result

In [ ]:
up_model = upsample(3, 4)
up_result = up_model(down_result)
print (up_result.shape)

In [ ]:
def Generator():
  inputs = tf.keras.layers.Input(shape=[256,256,3])

  down_stack = [
    downsample(64, 4, apply_batchnorm=False), # (bs, 128, 128, 64)
    downsample(128, 4), # (bs, 64, 64, 128)
    downsample(256, 4), # (bs, 32, 32, 256)
    downsample(512, 4), # (bs, 16, 16, 512)
    downsample(512, 4), # (bs, 8, 8, 512)
    downsample(512, 4), # (bs, 4, 4, 512)
    downsample(512, 4), # (bs, 2, 2, 512)
    downsample(512, 4), # (bs, 1, 1, 512)
  ]

  up_stack = [
    upsample(512, 4, apply_dropout=True), # (bs, 2, 2, 1024)
    upsample(512, 4, apply_dropout=True), # (bs, 4, 4, 1024)
    upsample(512, 4, apply_dropout=True), # (bs, 8, 8, 1024)
    upsample(512, 4), # (bs, 16, 16, 1024)
    upsample(256, 4), # (bs, 32, 32, 512)
    upsample(128, 4), # (bs, 64, 64, 256)
    upsample(64, 4), # (bs, 128, 128, 128)
  ]

  initializer = tf.random_normal_initializer(0., 0.02)
  last = tf.keras.layers.Conv2DTranspose(OUTPUT_CHANNELS, 4,
                                         strides=2,
                                         padding='same',
                                         kernel_initializer=initializer,
                                         activation='tanh') # (bs, 256, 256, 3)

  x = inputs

  # Downsampling through the model
  skips = []
  for down in down_stack:
    x = down(x)
    skips.append(x)

  skips = reversed(skips[:-1])

  # Upsampling and establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x)
    x = tf.keras.layers.Concatenate()([x, skip])

  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
generator = Generator()
tf.keras.utils.plot_model(generator, show_shapes=True, dpi=64)

In [ ]:
gen_output = generator(inp[tf.newaxis,...], training=False)
plt.imshow(gen_output[0,...])

- **ジェネレータの損失**
    - 生成された画像と **1 の配列**のシグモイド交差エントロピー損失です。
    - [論文](https://arxiv.org/abs/1611.07004)には、生成された画像とターゲット画像間の MAE（平均絶対誤差）である L1 損失も含まれます。
    - これにより、生成された画像は、構造的にターゲット画像に似るようになります。
    - 合計ジェネレータ損失の計算式は、gan_loss + LAMBDA * l1_loss で、LAMBDA = 100 です。この値は[論文](https://arxiv.org/abs/1611.07004)の執筆者が決定したものです。

以下に、ジェネレータのトレーニング手順を示します。

In [ ]:
LAMBDA = 100

In [ ]:
def generator_loss(disc_generated_output, gen_output, target):
  gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)

  # mean absolute error
  l1_loss = tf.reduce_mean(tf.abs(target - gen_output))

  total_gen_loss = gan_loss + (LAMBDA * l1_loss)

  return total_gen_loss, gan_loss, l1_loss

![Generator Update Image](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/generative/images/gen.png?raw=1)


## ディスクリミネータを構築する

- ディスクリミネータは PatchGAN です。
- ディスクリミネータの各ブロックは (Conv -> BatchNorm -> Leaky ReLU) です。
- 最後のレイヤーの後の出力の形状は (batch_size, 30, 30, 1) です。
- 出力の各 30x30 パッチは入力画像の 70x70 の部分を分類します（PatchGAN アーキテクチャの名前の由来です）。
- ディスクリミネータは 2 つの入力を受け取ります。
    - 入力画像とターゲット画像。本物として分類する画像です。
    - 入力画像と生成差r多画像（ジェネレータの出力）。偽物として分類する画像です。
    - これらの 2 つの入力をコードで連結します（`tf.concat([inp, tar], axis=-1)`）。

In [ ]:
def Discriminator():
  initializer = tf.random_normal_initializer(0., 0.02)

  inp = tf.keras.layers.Input(shape=[256, 256, 3], name='input_image')
  tar = tf.keras.layers.Input(shape=[256, 256, 3], name='target_image')

  x = tf.keras.layers.concatenate([inp, tar]) # (bs, 256, 256, channels*2)

  down1 = downsample(64, 4, False)(x) # (bs, 128, 128, 64)
  down2 = downsample(128, 4)(down1) # (bs, 64, 64, 128)
  down3 = downsample(256, 4)(down2) # (bs, 32, 32, 256)

  zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3) # (bs, 34, 34, 256)
  conv = tf.keras.layers.Conv2D(512, 4, strides=1,
                                kernel_initializer=initializer,
                                use_bias=False)(zero_pad1) # (bs, 31, 31, 512)

  batchnorm1 = tf.keras.layers.BatchNormalization()(conv)

  leaky_relu = tf.keras.layers.LeakyReLU()(batchnorm1)

  zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu) # (bs, 33, 33, 512)

  last = tf.keras.layers.Conv2D(1, 4, strides=1,
                                kernel_initializer=initializer)(zero_pad2) # (bs, 30, 30, 1)

  return tf.keras.Model(inputs=[inp, tar], outputs=last)

In [ ]:
discriminator = Discriminator()
tf.keras.utils.plot_model(discriminator, show_shapes=True, dpi=64)

In [ ]:
disc_out = discriminator([inp[tf.newaxis,...], gen_output], training=False)
plt.imshow(disc_out[0,...,-1], vmin=-20, vmax=20, cmap='RdBu_r')
plt.colorbar()

**ディスクリミネータの損失**

- ディスクリミネータの損失関数は、**本物の画像と生成された画像**の 2 つの入力を取ります。
- real_loss は**本物の画像**と**{nbsp}1 の配列（これらは本物の画像であるため）**のシグモイド交差エントロピー損失です。
- generated_loss は**生成された画像**と **0 の配列（偽物の画像であるため）**のシグモイド交差エントロピー損失です。
- total_loss は real_loss と generated_loss の和です。


In [ ]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def discriminator_loss(disc_real_output, disc_generated_output):
  real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)

  generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)

  total_disc_loss = real_loss + generated_loss

  return total_disc_loss

以下に、ディスクリミネータのトレーニング手順を示します。

このアーキテクチャとハイパーパラメータについては、[論文](https://arxiv.org/abs/1611.07004)をご覧ください。

![Generator Update Image](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/generative/images/gen.png?raw=1)


## オプティマイザとチェックポイントセーバーを定義する


In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

## 画像を生成する

トレーニング中に画像を描画する関数を記述します。

- テストデータセットからジェネレータに画像を渡します。
- ジェネレータは入力画像を出力に変換します。
- 最後に、予測を描画します。**出来上がり！**

注意: `training=True` は、テストデータセットでモデルを実行中にバッチ統計を行うために、ここに意図的に指定されています。training=False を使用した場合、トレーニングデータセットから学習した蓄積された統計が取得されます（ここでは使用されません）。

In [ ]:
def generate_images(model, test_input, tar):
  prediction = model(test_input, training=True)
  plt.figure(figsize=(15,15))

  display_list = [test_input[0], tar[0], prediction[0]]
  title = ['Input Image', 'Ground Truth', 'Predicted Image']

  for i in range(3):
    plt.subplot(1, 3, i+1)
    plt.title(title[i])
    # getting the pixel values between [0, 1] to plot it.
    plt.imshow(display_list[i] * 0.5 + 0.5)
    plt.axis('off')
  plt.show()

In [ ]:
for example_input, example_target in test_dataset.take(1):
  generate_images(generator, example_input, example_target)

## トレーニング

- 各サンプルについて、入力は出力を生成します。
- ディスクリミネータは input_image と生成された画像を最初の入力として受け取ります。2 番目の入力は input_image と target_image です。
- 次に、ジェネレータとディスクリミネータの損失を計算します。
- ジェネレータとディスクリミネータの変数（入力）の両方に関して損失の勾配を計算し、これらをオプティマイザに適用します。
- そして、損失を TensorBoard にログします。

In [ ]:
EPOCHS = 150

In [ ]:
import datetime
log_dir="logs/"

summary_writer = tf.summary.create_file_writer(
  log_dir + "fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [ ]:
@tf.function
def train_step(input_image, target, epoch):
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    gen_output = generator(input_image, training=True)

    disc_real_output = discriminator([input_image, target], training=True)
    disc_generated_output = discriminator([input_image, gen_output], training=True)

    gen_total_loss, gen_gan_loss, gen_l1_loss = generator_loss(disc_generated_output, gen_output, target)
    disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

  generator_gradients = gen_tape.gradient(gen_total_loss,
                                          generator.trainable_variables)
  discriminator_gradients = disc_tape.gradient(disc_loss,
                                               discriminator.trainable_variables)

  generator_optimizer.apply_gradients(zip(generator_gradients,
                                          generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients,
                                              discriminator.trainable_variables))

  with summary_writer.as_default():
    tf.summary.scalar('gen_total_loss', gen_total_loss, step=epoch)
    tf.summary.scalar('gen_gan_loss', gen_gan_loss, step=epoch)
    tf.summary.scalar('gen_l1_loss', gen_l1_loss, step=epoch)
    tf.summary.scalar('disc_loss', disc_loss, step=epoch)

実際のトレーニングループは以下のようになります。

- エポックの回数をイテレートします。
- エポックごとに表示を消去し、`generate_images` を実行してその進行状況を表示します。
- エポックごとにトレーニングデータセットをイテレートし、各サンプルに '.' を出力します。
- 20 エポックごとにチェックポイントが保存されます。

In [ ]:
def fit(train_ds, epochs, test_ds):
  for epoch in range(epochs):
    start = time.time()

    display.clear_output(wait=True)

    for example_input, example_target in test_ds.take(1):
      generate_images(generator, example_input, example_target)
    print("Epoch: ", epoch)

    # Train
    for n, (input_image, target) in train_ds.enumerate():
      print('.', end='')
      if (n+1) % 100 == 0:
        print()
      train_step(input_image, target, epoch)
    print()

    # saving (checkpoint) the model every 20 epochs
    if (epoch + 1) % 20 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                        time.time()-start))
  checkpoint.save(file_prefix = checkpoint_prefix)

このトレーニングループは、TensorBoard で簡単に閲覧し、トレーニングの進行状況を監視できるログを保存します。ローカル環境で動作する場合は、別の tensorboard プロセスを起動してください。ノートパソコンで TensorBoard による監視を行う場合は、トレーニングを開始する前にビューアを起動しておくことが最も簡単な方法です。

ビューアを起動するには、以下ののコードをコードセルに貼り付けます。

In [ ]:
#docs_infra: no_execute
%load_ext tensorboard
%tensorboard --logdir {log_dir}

トレーニングループを実行します。

In [ ]:
fit(train_dataset, EPOCHS, test_dataset)

TensorBoard の結果を*公開*共有する場合は、以下のコードをコードセルに張り付けて、[TensorBoard.dev](https://tensorboard.dev/) にログをアップロードできます。

注意: Google アカウントが必要です。

```
!tensorboard dev upload --logdir  {log_dir}
```

要注意: このコマンドは終了しません。長時間に及ぶ実験の結果を連続的にアップロードするように設計されています。データのアップロードが完了したら、ノートブックツールの "interrupt execution" オプションを使って停止する必要があります。

[TensorBoard.dev](https://tensorboard.dev/) で、このノートブックの[前回の実行の結果](https://tensorboard.dev/experiment/lZ0C6FONROaUMfjYkVyJqw)を閲覧できます。

TensorBoard.dev は、ML の実験をホスト、追跡、および共有するための、公開マネージドエクスペリエンスです。

また、`<iframe>` を使用してインラインに含めることもできます。

In [ ]:
display.IFrame(
    src="https://tensorboard.dev/experiment/lZ0C6FONROaUMfjYkVyJqw",
    width="100%",
    height="1000px")

GAN のログは、単純な分類または回帰モデルよりもわずかにしか解釈できません。以下の項目を特定してください。

- いずれのモデルにも "won" がないことを確認してください。`gen_gan_loss` または `disc_loss` のいずれかが非常に低い場合、そのモデルがもう片方のモデルを宇和待っていることを示しているため、混合モデルを正しくトレーニングできていないことになります。
- 値 `log(2) = 0.69` は、これらの損失の適切な基準点です。パープレキシティ（予測性能）が 2 であるということは、ディスクリミネータが、平均して 2 つのオプションについて等しく不確実であることを表します。
- `disc_loss` については、値が `0.69` を下回る場合、ディスクリミネータは、実際の画像と生成された画像を組み合わせたセットにおいて、ランダムよりも優れていることを示します。
- `gen_gan_loss` については、値が `0.69` を下回る場合、ディスクリミネータを騙す上で、ジェネレータがランダムよりも優れていることを示します。
- トレーニングが進行するにつれ、`gen_l1_loss` は下降します。

## 最新のチェックポイントを復元してテストする

In [ ]:
!ls {checkpoint_dir}

In [ ]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

## テストデータセットを使用して生成する

In [ ]:
# Run the trained model on a few examples from the test dataset
for inp, tar in test_dataset.take(5):
  generate_images(generator, inp, tar)